In [16]:
from pathlib import Path
import requests
import zipfile

In [17]:
data_path = Path("data/raw")
dataset_path = data_path / "ir_dataset"
if dataset_path.is_dir():
    print(f"{dataset_path} already exists.. skipping download")
else:
    print(f"{dataset_path} does not exist.. downloading")
    dataset_path.mkdir(parents=True, exist_ok=True)

    # Download the dataset
    with open(data_path / "ir-anthology-v1.zip", "wb") as f:
        url = "https://files.webis.de/teaching/ir-ss23/ir-anthology-v1.zip"
        request = requests.get(url)
        print("Downloading data...")
        f.write(request.content)

    with zipfile.ZipFile(data_path / "ir-anthology-v1.zip", "r") as zip_ref:
        print("Unzipping..")
        zip_ref.extractall(dataset_path)
        print("Done!")


data/raw/ir_dataset does not exist.. downloading
Unzipping..
Done!


In [18]:
import jsonlines
import pprint

with jsonlines.open(dataset_path / "ir-anthology-07-11-2021-ss23.jsonl", 'r') as jsonl_f:
     lst = [obj for obj in jsonl_f]
     # only keep the keys "id" and "title"
     lst = [{k: v for k, v in obj.items() if k in ['id', 'title']} for obj in lst]
     # rename the id key to doc_id and the title key to text
     for json_elem in lst:
         json_elem['doc_id'] = json_elem.pop('id')
         json_elem['text'] = json_elem.pop('title')
     pprint.pprint(lst[0])


{'doc_id': '2019.sigirconf_workshop-2019birndl.0',
 'text': 'Proceedings of the 4th Joint Workshop on Bibliometric-enhanced '
         'Information Retrieval and Natural Language Processing for Digital '
         'Libraries (BIRNDL 2019) co-located with the 42nd International ACM '
         'SIGIR Conference on Research and Development in Information '
         'Retrieval (SIGIR 2019), Paris, France, July 25, 2019'}


In [19]:
processed_path = Path("data/processed")
processed_path.mkdir(parents=True, exist_ok=True)

# write the processed data to a jsonl file
with jsonlines.open(processed_path / 'output.jsonl', 'w') as writer:
    writer.write_all(lst)